In [ ]:
pip install --upgrade openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
import pandas as pd
import regex as re
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data=pd.read_csv('/content/Implicit_TE_GPT3_5_GPT4_prompt.csv')
print(len(data))
data.head(2)

23266


,event_story,answer,Event,Context,key,GPT3,GPT4
0,event: The teacher asked us to stop talking st...,positive,The teacher asked us to stop talking starts a...,I was so nervous for my first day of school. ...,TRACIE,True,False
1,event: The teacher asked us to stop talking st...,negative,The teacher asked us to stop talking starts b...,I was so nervous for my first day of school. ...,TRACIE,False,False


In [ ]:
data.answer.value_counts()

 positive    11633
 negative    11633
Name: answer, dtype: int64

In [ ]:
#only extract the positive relations to use it in QA format
MT_QA= data[data.answer==' positive']
print(len(MT_QA))
MT_QA.head(2)

11633


,event_story,answer,Event,Context,key,GPT3,GPT4
0,event: The teacher asked us to stop talking st...,positive,The teacher asked us to stop talking starts a...,I was so nervous for my first day of school. ...,TRACIE,True,False
2,event: The man was holding a piece of paper st...,positive,The man was holding a piece of paper starts b...,"John didn't know he was being followed. ""When...",TRACIE,True,True


In [ ]:
#converting starts after|ends after as after and starts before|ends before as before
#Extracting the Actual Relation, event 1 and event 2 from the event
MT_QA['Event']=MT_QA['Event'].str.lower()
MT_QA['event1']=MT_QA['Event'].apply(lambda x: re.split(r'starts after|starts before|ends after|ends before', x)[0])
MT_QA['event2']=MT_QA['Event'].apply(lambda x: re.split(r'starts after|starts before|ends after|ends before', x)[1])
MT_QA['Relation']=MT_QA['Event'].apply(lambda x: 'after' if x.find('starts after')>-1 else(
                                  'before' if x.find('starts before')>-1  else (
                                 'after' if x.find('ends after')>-1 else 'before')))

In [ ]:
MT_QA['Relation'].value_counts()

before    6722
after     4911
Name: Relation, dtype: int64

In [ ]:
MT_QA.reset_index(drop=True, inplace=True)
MT_QA

,event_story,answer,Event,Context,key,GPT3,GPT4,event1,event2,Relation
0,event: The teacher asked us to stop talking st...,positive,the teacher asked us to stop talking starts a...,I was so nervous for my first day of school. ...,TRACIE,True,False,the teacher asked us to stop talking,we talked the whole time during and after cla...,after
1,event: The man was holding a piece of paper st...,positive,the man was holding a piece of paper starts b...,"John didn't know he was being followed. ""When...",TRACIE,True,True,the man was holding a piece of paper,he hand john a small piece of paper,before
2,event: Cecily had an upset stomach starts afte...,positive,cecily had an upset stomach starts after the ...,"""After the three day music festival, Cecily w...",TRACIE,False,False,cecily had an upset stomach,the music festival,after
3,event: Paul is not friendly. starts after Paul...,positive,paul is not friendly. starts after paul hat h...,Paul hates his job. Everyday at work he gets ...,TRACIE,True,True,paul is not friendly.,paul hat his job,after
4,event: Noah realized that kayaking was not fun...,positive,noah realized that kayaking was not fun start...,Noah thought kayaking looked fun. He rented a...,TRACIE,True,True,noah realized that kayaking was not fun,he discovered that kayaking was harder than i...,after
...,...,...,...,...,...,...,...,...,...,...
11628,event: a 52-year - old obstetrician and gyneco...,positive,a 52-year - old obstetrician and gynecologist...,Federal law enforcement officials said Wednes...,MATRESS,False,True,a 52-year - old obstetrician and gynecologist...,"slepian , a 52-year - old obstetrician and gy...",before
11629,event: a 52-year - old obstetrician and gyneco...,positive,a 52-year - old obstetrician and gynecologist...,Federal law enforcement officials said Wednes...,MATRESS,False,False,a 52-year - old obstetrician and gynecologist...,n.y. chatting with his wife and one of his fo...,before
11630,"event: Slepian , a 52-year - old obstetrician ...",positive,"slepian , a 52-year - old obstetrician and gy...",Federal law enforcement officials said Wednes...,MATRESS,True,False,"slepian , a 52-year - old obstetrician and gy...",he stood in the kitchen of his home in suburb...,after
11631,event: he stood in the kitchen of his home in ...,positive,he stood in the kitchen of his home in suburb...,_ officials said they had decided to issue a ...,MATRESS,False,False,he stood in the kitchen of his home in suburb...,n.y. chatting with his wife and one of his fo...,before


#GPT 3.5 Turbo -1106

In [ ]:
from openai import OpenAI
def get_completion(prompt, model='gpt-4-1106-preview'): #change the model version as per your needs
  client = OpenAI(api_key = '') # your api key here
  messages=[{"role":"user","content":prompt}]
  response=client.chat.completions.create(model=model,messages=messages,temperature=0)
  return response.choices[0].message.content

In [ ]:
data['GPT3_5_QA']=''
data['GPT4_QA']=''

In [ ]:
prediction=[]
print(start_index)
for index,row in MT_QA.iterrows():
    # event1='('+str(row.e1id)+':'+str(row.event1)+')'
    # event2='('+str(row.e2id)+':'+str(row.event2)+')'
    prompt=  "context:'{0}'.\nE1:'{1}'.\nE2:'{2}'.\nFor the given context, give the temporal relation between E1 and E2 as before or after. Give your answer in the format, Relation:<ans>".format(row.Context,row.event1,row.event2)
    response = get_completion(prompt)
    print(prompt)
    print(response)
    prediction.append(re.findall("^Relation:(.*\w*)", response)[0] if len(re.findall("^Relation:(.*\w*)", response))>0 else 'None')

0
context:' I was so nervous for my first day of school. "When I arrived at my first class, I was shaking." I sat next the prettiest girl in class. We talked the whole time during and after class. "We became best friends, and she made college so much easier!"'.
E1:' the teacher asked us to stop talking '.
E2:' we talked the whole time during and after class. '.
For the given context, give the temporal relation between E1 and E2 as before or after. Give your answer in the format, Relation:<ans>
Relation: before
context:' John didn't know he was being followed. "When he got on the train, a man approached him." He handed John a small piece of paper. It said RAVINCORP in bold letters. "John had been offered a job by the CIA, he just didn't know it yet."'.
E1:' the man was holding a piece of paper '.
E2:' he hand john a small piece of paper '.
For the given context, give the temporal relation between E1 and E2 as before or after. Give your answer in the format, Relation:<ans>
Relation: befo

KeyboardInterrupt: ignored

In [ ]:
MT_QA['GPT4_QA']=prediction
print(len(prediction))

1574


In [ ]:
MT_QA.to_csv('all_data_QA_both.csv', index=False, header=False,mode='a')

In [ ]:
res=pd.read_csv('/content/all_data_QA_both.csv')
res.drop(['Unnamed: 0'],axis=1,inplace=True)
print(len(res))
res.head(2)

1576


,key,context,e1id,event1,e1tense,e2id,event2,e2tense,verb_tense,event1_pos,no_of_events,word_count,sent_count,Relation,prediction,GPT4,GPT3_5_QA,GPT4_QA
0,AQUAINT,APW19980807.0261 NEWS STORY 08/07/1998 06:1...,1,exploded,PAST,2,killing,PRESPART,DIFFERENT,1,3,75,2,BEFORE,AFTER,BEFORE,after,before
1,AQUAINT,embassies in the Kenyan and Tanzanian capitals...,2,killing,PRESPART,3,said,PAST,DIFFERENT,1,2,13,1,BEFORE,AFTER,BEFORE,before,before


#GPT 4

In [ ]:
prediction=[]
print(start_index)
for index,row in res.iterrows():
    prompt= "context:'{0}'.\nE1:'{1}'.\nE2:'{2}'.\nFor the given context, give the temporal relation between E1 and E2 as before or after. Give your answer in the format, Relation:<ans>".format(row.Context,row.event1,row.event2)
    response = get_completion(prompt)
    print(response)
    prediction.append(re.findall("^Relation:(.*\w*)", response)[0] if len(re.findall("^Relation:(.*\w*)", response))>0 else 'None')

11077
Relation: after
Relation: after
Relation: before
Relation: after
Relation: after
Relation:before
Relation: before
Relation: after
Relation:before
Relation:before
Relation: after
Relation: before
Relation: before
Relation: before
Relation: after
Relation: after
Relation: before
Relation: before
Relation: after
Relation: after
Relation: before
Relation: before
Relation: after
Relation: before
Relation: before
Relation: before
Relation: after
Relation: before
Relation: after
Relation: before
Relation: after
Relation: before
Relation: after
Relation: after
Relation: after
Relation: after
Relation: after
Relation:after
Relation:after
Relation: before
Relation: after
Relation:after
Relation: before
Relation:before
Relation:before
Relation:before
Relation: after
Relation: before
Relation: after
Relation: before
Relation:before
Relation: before
Relation: before
Relation: after
Relation: after
Relation: after
Relation:after
Relation: after
Relation: after
Relation: after
Relation: after
R

In [ ]:
res['GPT4_QA']=prediction
print(len(prediction))

575


In [ ]:
res.to_csv('Implicit_QA_both_GPT_final.csv', index=False, header=False,mode='a')